# Comparison to DuPLO Model


Train models via 
```bash
python train_duplo.py 23classes GAFv2
python train_duplo.py 12classes GAFv2
python train_duplo.py 23classes BavarianCrops
python train_duplo.py 12classes BavarianCrops
```

In [3]:
import sys
sys.path.append("../src")
sys.path.append("../src/models")

from models.duplo import DuPLO
import torch
from train import prepare_dataset
from argparse import Namespace
from tqdm.notebook import tqdm
import numpy as np
import sklearn.metrics
import pandas as pd
import os
from datasets.BavarianCrops_Dataset import BavarianCropsDataset
from train_duplo import metrics

def merge(namespaces):
    merged = dict()

    for n in namespaces:
        d = n.__dict__
        for k,v in d.items():
            merged[k]=v

    return Namespace(**merged)

TUM_dataset = Namespace(
    dataset = "BavarianCrops",
    trainregions = ["holl","nowa","krum"],
    testregions = ["holl","nowa","krum"],
    scheme="blocks",
    test_on = "test",
    train_on = "trainvalid",
    samplet = 70
)

GAF_dataset = Namespace(
    dataset = "GAFv2",
    trainregions = ["holl","nowa","krum"],
    testregions = ["holl","nowa","krum"],
    features = "optical",
    scheme="blocks",
    test_on="test",
    train_on="train",
    samplet = 23
)

def setup(dataset, mode, dataroot="../data", store = '/tmp/'):
    
    if mode == "12classes":
        classmapping = os.path.join(dataroot,dataset,'classmapping12.csv')
    elif mode == "23classes":
        classmapping = os.path.join(dataroot,dataset,'classmapping23.csv')
    
    args = Namespace(batchsize=256,
                 classmapping=classmapping,
                 dataroot=dataroot, dataset=dataset,
                 model='duplo',mode=None,
                 seed=0, store=store, workers=0)

    if dataset == "BavarianCrops":
        args = merge([args,TUM_dataset])
        exp = "isprs_tum_duplo"
    elif dataset == "GAFv2":
        args = merge([args,GAF_dataset])
        exp = "isprs_gaf_duplo"
        
    traindataloader, testdataloader = prepare_dataset(args)
        
    input_dim = traindataloader.dataset.datasets[0].ndims
    nclasses = len(traindataloader.dataset.datasets[0].classes)

    device = torch.device("cuda")
    model = DuPLO(input_dim=input_dim, nclasses=nclasses, sequencelength=args.samplet, dropout=0.4)
    model.load(f"/tmp/{mode}/{exp}/model.pth")

    model.to(device)

    
    return testdataloader, model

def evaluate(model, dataloader):
    stats = list()
    model.cuda()
    model.eval()
    y_preds = list()
    ys = list()
    ids = list()
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(dataloader),total=len(dataloader)):
            X,y,id = batch
            logsoftmax , *_ = model.forward(X.transpose(1,2).cuda())
            y_pred = logsoftmax.argmax(dim=1)
            ys.append(y.cpu().detach().numpy())
            y_preds.append(y_pred.cpu().detach().numpy())
            ids.append(id)
    model.cpu()
    return np.hstack(y_preds), np.vstack(ys)[:,0], np.hstack(ids)

## TUM Dataset 

### 12 classes

In [5]:
testdataloader, model = setup("BavarianCrops","12classes")
y_pred, y, ids = evaluate(model, testdataloader)
print(sklearn.metrics.classification_report(y,y_pred))
metrics(y,y_pred)

Initializing BavarianCropsDataset test partition in holl
read 12 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping12.csv/blocks/holl/test
loaded 9583 samples
Dataset ../data/BavarianCrops. region holl. partition test. X:9583x(144, 13), y:(9583,) with 12 classes
Initializing BavarianCropsDataset test partition in nowa
read 12 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping12.csv/blocks/nowa/test
loaded 3547 samples
Dataset ../data/BavarianCrops. region nowa. partition test. X:3547x(287, 13), y:(3547,) with 12 classes
Initializing BavarianCropsDataset test partition in krum
read 12 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping12.csv/blocks/krum/test
loaded 4278 samples
Dataset ../data/BavarianCrops. region krum. partition test. X:4278x(143, 13), y:(4278,) with 12 classes
Initializing BavarianCropsDataset trainvalid partition in holl
read 12 classes
precached dataset files found at ../data/Bava


              precision    recall  f1-score   support

           0       0.55      0.37      0.44       581
           1       0.95      0.97      0.96     10878
           2       0.59      0.84      0.69       995
           3       0.93      0.96      0.95      1917
           4       0.00      0.00      0.00        56
           5       0.50      0.02      0.04        52
           6       0.00      0.00      0.00       104
           7       0.68      0.74      0.71       777
           8       0.77      0.83      0.80      1017
           9       0.57      0.28      0.37       275
          10       0.48      0.17      0.26       499
          11       0.96      0.82      0.89       257

    accuracy                           0.87     17408
   macro avg       0.58      0.50      0.51     17408
weighted avg       0.86      0.87      0.86     17408



{'accuracy': 0.8741383272058824,
 'kappa': 0.7815129422964229,
 'f1_micro': 0.8741383272058825,
 'f1_macro': 0.5078856649252542,
 'f1_weighted': 0.8609462493356634,
 'recall_micro': 0.8741383272058824,
 'recall_macro': 0.5001654501970126,
 'recall_weighted': 0.8741383272058824,
 'precision_micro': 0.8741383272058824,
 'precision_macro': 0.5812256570424775,
 'precision_weighted': 0.8591904981624588}

### 23 classes

In [6]:
testdataloader, model = setup("BavarianCrops","23classes")
y_pred, y, ids = evaluate(model, testdataloader)
print(sklearn.metrics.classification_report(y,y_pred))
metrics(y,y_pred)

Initializing BavarianCropsDataset test partition in holl
read 23 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping23.csv/blocks/holl/test
loaded 9792 samples
Dataset ../data/BavarianCrops. region holl. partition test. X:9792x(144, 13), y:(9792,) with 23 classes
Initializing BavarianCropsDataset test partition in nowa
read 23 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping23.csv/blocks/nowa/test
loaded 3572 samples
Dataset ../data/BavarianCrops. region nowa. partition test. X:3572x(287, 13), y:(3572,) with 23 classes
Initializing BavarianCropsDataset test partition in krum
read 23 classes
precached dataset files found at ../data/BavarianCrops/npy/classmapping23.csv/blocks/krum/test
loaded 4306 samples
Dataset ../data/BavarianCrops. region krum. partition test. X:4306x(143, 13), y:(4306,) with 23 classes
Initializing BavarianCropsDataset trainvalid partition in holl
read 23 classes
precached dataset files found at ../data/Bava


              precision    recall  f1-score   support

           0       0.19      0.07      0.10       356
           1       0.46      0.68      0.55       225
           2       0.65      0.29      0.40       882
           3       0.00      0.00      0.00       469
           4       0.00      0.00      0.00        23
           5       0.65      0.78      0.71       995
           6       0.92      0.96      0.94      1917
           7       0.00      0.00      0.00        56
           8       0.50      0.06      0.11        16
           9       0.38      0.08      0.14       121
          10       0.84      0.97      0.90      9301
          11       0.25      0.00      0.01       226
          12       0.00      0.00      0.00        52
          13       0.27      0.12      0.17       104
          14       0.60      0.84      0.70       777
          15       0.74      0.82      0.78      1017
          16       0.53      0.38      0.44       248
          17       0.46   

{'accuracy': 0.7900396151669496,
 'kappa': 0.6795626476440702,
 'f1_micro': 0.7900396151669495,
 'f1_macro': 0.342844877582905,
 'f1_weighted': 0.747589372509714,
 'recall_micro': 0.7900396151669496,
 'recall_macro': 0.3417897309311548,
 'recall_weighted': 0.7900396151669496,
 'precision_micro': 0.7900396151669496,
 'precision_macro': 0.4343676608558107,
 'precision_weighted': 0.73517078259069}

## GAF Dataset

### 12 classes

In [23]:
testdataloader, model = setup("GAFv2","12classes")
y_pred, y, ids = evaluate(model, testdataloader)
print(sklearn.metrics.classification_report(y,y_pred))
metrics(y,y_pred)

5261156

### 23 classes

In [68]:
testdataloader, model = setup("GAFv2","23classes")
y_pred, y, ids = evaluate(model, testdataloader)
print(sklearn.metrics.classification_report(y,y_pred))
metrics(y,y_pred)

array([1, 1, 8, ..., 1, 1, 1])